In [ ]:
# https://ourworldindata.org/grapher/gdp-world-regions-stacked-area
import pandas as pd

df_pib = pd.read_csv("gdp-world-regions-stacked-area.csv")
df_pib

In [ ]:
df_pib_sum = df_pib.groupby("Year").sum().reset_index()
df_pib_world = df_pib.query("Entity == 'World'")
df_pib_world

In [ ]:
import plotly.express as px

fig = px.line(df_pib_world, x="Year", y="GDP", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
# https://ourworldindata.org/grapher/population-since-1800
import pandas as pd

df_population = pd.read_csv("population-since-1800.csv")
df_population

In [ ]:
df_population_world = df_population.query("Entity == 'World'")
df_population_world

In [ ]:
import plotly.express as px

fig = px.line(
    df_population_world,
    x="Year",
    y="Population (historical estimates)",
    width=800,
    height=800,
)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
# https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html

from shifter_pandas.bp import UNITS_ENERGY, BPDatasource

shifter_ds = BPDatasource("bp-stats-review-2021-all-data.xlsx")

In [ ]:
for type_ in shifter_ds.metadata():
    print(f"{type_['label']} [{type_['unit']}]")

In [ ]:
df = shifter_ds.datasource(
    regions_filter=["Total World"],
    types_filter=[
        "Primary Energy Consumption",
        "CO2 From Flaring",
        "CO2 Excluding Flaring",
    ],
)
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="year", y="value", color="type unit", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
df_co2 = (
    df.query("type in ['CO2 From Flaring', 'CO2 Excluding Flaring']")
    .groupby(["year"])
    .sum()
    .reset_index()
)
df_co2

In [ ]:
import plotly.express as px

fig = px.line(df_co2, x="year", y="value", width=800, height=800)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()

In [ ]:
df_co2_pib = pd.merge(
    df_co2,
    df_pib_world,
    how="inner",
    on=None,
    left_on="year",
    right_on="Year",
)
df_co2_pib

In [ ]:
import plotly.express as px

fig = px.scatter(df_co2_pib, x="value", y="GDP", width=800, height=800, trendline="ols")
fig.update_layout()
fig.show()

In [ ]:
df_e_pib = pd.merge(
    df.query('type == "Primary Energy Consumption"'),
    df_pib_world,
    how="inner",
    on=None,
    left_on="year",
    right_on="Year",
)
df_e_pib

In [ ]:
import plotly.express as px

fig = px.scatter(df_co2_pib, x="value", y="GDP", width=800, height=800, trendline="ols")
fig.update_layout()
fig.show()

In [ ]:
df_co2_pib2 = df_co2_pib.assign(co2_pib=df_co2_pib.value / df_co2_pib.GDP)
df_co2_pib2

In [ ]:
import plotly.express as px

fig = px.line(df_co2_pib2, x="year", y="co2_pib", width=800, height=800)
fig.update_layout()
fig.show()